# Lab 02: Credit Ratings

---
author: Alejandra Arias
date: September 13, 2024
embed-resources: true
---

## Introduction

## Methods

In [1]:
# imports
import pandas as pd

### Data

In [5]:
# load data
credit_train = pd.read_csv("https://cs307.org/lab-02/data/credit-train.csv")
credit_test = pd.read_csv("https://cs307.org/lab-02/data/credit-test.csv")
credit_train

,Rating,Income,Age,Education,Gender,Student,Married,Ethnicity
0,257.0,44.473,81.0,16.0,Female,No,No,NaN
1,353.0,41.532,50.0,NaN,Male,No,Yes,Caucasian
2,388.0,16.479,26.0,16.0,Male,NaN,No,NaN
3,321.0,10.793,29.0,13.0,Male,No,No,Caucasian
4,367.0,76.273,65.0,14.0,Female,No,Yes,Caucasian
...,...,...,...,...,...,...,...,...
251,268.0,26.370,78.0,11.0,Male,No,Yes,Asian
252,433.0,26.427,50.0,15.0,Female,Yes,Yes,Asian
253,259.0,12.031,58.0,18.0,Female,NaN,Yes,Caucasian
254,335.0,80.861,29.0,15.0,Female,No,Yes,Asian


In [4]:
# summary statistics
samples = credit_train.shape[0]
features = credit_train.shape[1] - 1
samples, features

(256, 7)

In [22]:
rating = credit_train["Rating"].mean()
ratstd = credit_train["Rating"].std()
rating, ratstd

(347.609375, 148.8931046679028)

In [9]:
rating_counts = credit_train[["Rating", "Student"]].groupby("Student").count()
rating_mean = credit_train["Rating"].groupby(credit_train["Student"]).mean()
rating_std = credit_train["Rating"].groupby(credit_train["Student"]).std()
rating_counts, rating_mean, rating_std

(         Rating
 Student        
 No          213
 Yes          19,
 Student
 No     349.511737
 Yes    338.473684
 Name: Rating, dtype: float64,
 Student
 No     148.003525
 Yes    149.164476
 Name: Rating, dtype: float64)

In [11]:
marriage_counts = credit_train[["Married", "Rating"]].groupby("Married").count()
marriage_mean = credit_train["Rating"].groupby(credit_train["Married"]).mean()
marriage_std = credit_train["Rating"].groupby(credit_train["Married"]).std()
marriage_counts, marriage_mean, marriage_std

(         Rating
 Married        
 No           98
 Yes         158,
 Married
 No     329.397959
 Yes    358.905063
 Name: Rating, dtype: float64,
 Married
 No     132.039649
 Yes    157.797935
 Name: Rating, dtype: float64)

In [17]:
income_correlation = credit_train[["Income", "Rating"]].corr().iloc[0, 1]
income_correlation

0.7708422040157111

In [ ]:
age_correlation = credit_train["Rating"].corr(credit_train["Age"])
age_correlation

In [28]:
credit_train["Age"].head(15)

0     81.0
1     50.0
2     26.0
3     29.0
4     65.0
5     77.0
6     38.0
7     35.0
8     77.0
9     67.0
10    58.0
11     NaN
12    73.0
13    31.0
14    79.0
Name: Age, dtype: float64

In [29]:
rating_missing = credit_train["Rating"].isnull().sum() / samples
income_missing = credit_train["Income"].isnull().sum() / samples
age_missing = credit_train["Age"].isnull().sum() /  samples
education_missing = credit_train["Education"].isnull().sum() / samples
gender_missing = credit_train["Gender"].isnull().sum() / samples
student_missing = credit_train["Student"].isnull().sum() /  samples
married_missing = credit_train["Married"].isnull().sum() / samples
ethnicitiy_missing = credit_train["Ethnicity"].isnull().sum() / samples
rating_missing, income_missing, age_missing, education_missing, gender_missing, student_missing, married_missing, ethnicitiy_missing

(0.0, 0.0, 0.12890625, 0.09765625, 0.0234375, 0.09375, 0.0, 0.09765625)

In [ ]:
# visualizations

### Models

In [ ]:
# process data for ML
# create X and y for train
X_train = credit_train.drop("Rating", axis=1)
y_train = credit_train["Rating"]

# create X and y for test
X_test = credit_test.drop("Rating", axis=1)
y_test = credit_test["Rating"]

In [ ]:
# train models

## Results

In [ ]:
# report model metrics

## Discussion

### Conclusion